In [22]:
import bt
import pandas as pd
import talib as ta

# List of stock symbols to test

stock_symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META']

data = bt.get(stock_symbols, start='2024-01-01', end='2024-5-21')

# Fill NaN values with zero

data_filled = data.fillna(0)

# Calculate RSI for each stock

rsi = data_filled.apply(lambda x: ta.RSI(x, timeperiod=14))

# Calculate ROC for each stock

roc = data_filled.apply(lambda x: ta.ROC(x, timeperiod=25))
#roc = data_filled.pct_change(25) * 100 # 25-day ROC, multiplied by 100 for percentage

# Create signals based on RSI and momentum

In [15]:
import dtale

In [16]:
dtale.show(rsi)

Exception on /dtale/main/highLightTitle.png [GET]
Traceback (most recent call last):
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/dtale/views.py", line 1238, in view_main
    if not global_state.contains(data_id):
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site

2024-05-20 17:22:32,824 - ERROR    - Exception on /dtale/main/highLightTitle.png [GET]
Traceback (most recent call last):
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/zlying/opt/miniconda3/envs/py310/lib/python3.10/site-packages/dtale/views.py", line 1238, in view_main
    if not global_state.contains(data_id):
  File "/Users/zlying/opt/mini

In [24]:
rsi_signal = rsi < 30

# Create a signal based on ROC

roc_signal = roc > 5

# Create the combined signal

combined_signal = rsi_signal 

# Create the strategy

my_strategy = bt.Strategy('s1', [
    bt.algos.RunAfterDate('2023-07-01'), # Run after the specified date
    bt.algos.RunWeekly(),
    bt.algos.SelectWhere(combined_signal),
    #bt.algos.SelectMomentum(n=1, lookback=pd.DateOffset(days=25)),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
])

# Create the backtest and run it

test = bt.Backtest(my_strategy, data)
result = bt.run(test)

# Access RSI values

rsi_values_list = []
roc_values_list = []

# Get the latest transactions

latest_transactions = result.get_transactions().tail(10) # You can adjust the number as needed

# Print RSI values for the stocks in the latest transactions

#for stock, date in latest_transactions.index:
for date, stock in latest_transactions.index:
    rsi_value = rsi.loc[date, stock]
    roc_value = roc.loc[date, stock]
    print(f"Stock: {stock}, Date: {date}, RSI: {rsi_value}, ROC: {roc_value}")

print("end")

Stock: aapl, Date: 2024-03-04 00:00:00, RSI: 29.72647350317042, ROC: -8.885026023456376
Stock: aapl, Date: 2024-03-11 00:00:00, RSI: 34.68239514317925, ROC: -6.93008904361564
Stock: meta, Date: 2024-04-29 00:00:00, RSI: 29.676874050816775, ROC: -15.102632262455252
Stock: meta, Date: 2024-05-06 00:00:00, RSI: 46.69969559880495, ROC: -5.224384473158283
end


In [27]:
result.get_transactions()

,,price,quantity
Date,Security,,
2024-03-04,aapl,174.862823,5718.0
2024-03-11,aapl,172.516006,-5718.0
2024-04-29,meta,432.619995,2280.0
2024-05-06,meta,465.679993,-2280.0


In [20]:
result.display()

Stat                 s1
-------------------  ----------
Start                2024-01-01
End                  2024-05-17
Risk-free rate       0.00%

Total Return         6.20%
Daily Sharpe         1.81
Daily Sortino        3.39
CAGR                 17.38%
Max Drawdown         -3.48%
Calmar Ratio         4.99

MTD                  8.25%
3m                   6.20%
6m                   -
YTD                  6.20%
1Y                   -
3Y (ann.)            -
5Y (ann.)            -
10Y (ann.)           -
Since Incep. (ann.)  17.38%

Daily Sharpe         1.81
Daily Sortino        3.39
Daily Mean (ann.)    16.18%
Daily Vol (ann.)     8.94%
Daily Skew           1.41
Daily Kurt           18.66
Best Day             3.04%
Worst Day            -2.84%

Monthly Sharpe       1.23
Monthly Sortino      8.65
Monthly Mean (ann.)  19.04%
Monthly Vol (ann.)   15.51%
Monthly Skew         1.91
Monthly Kurt         3.71
Best Month           8.25%
Worst Month          -1.34%

Yearly Sharpe        -
Yearly Sor